In [1]:
from constants import Split
from data.datastructures.hyperparameters.dpr import DenseHyperParams
from retriever.Contriever import Contriever
from data.loaders.RetrieverDataset import RetrieverDataset
from metrics.SimilarityMatch import CosineSimilarity as CosScore

In [2]:
loader = RetrieverDataset("wikimultihopqa", "wiki-musiqueqa-corpus", "evaluation/config.ini", Split.DEV)
queries, qrels, corpus = loader.qrels()

print("info: ", len(queries), len(qrels), len(corpus), queries[0])

c:\Users\alexm\miniforge3\envs\bcqa\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Transforming passage dataset: 100%|██████████| 563424/563424 [00:01<00:00, 344945.09it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\alexm\miniforge3\envs\bcqa\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='long

Harley-Davidson Harley-Davidson
KeysView(<Section: Data-Path>)
12576


100%|██████████| 1200/1200 [03:23<00:00,  5.90it/s]


Dataset loaded of length 12000
info:  1200 1200 563424 <data.datastructures.question.Question object at 0x000001DD9C567760>


In [8]:
config_instance = DenseHyperParams(query_encoder_path="facebook/contriever",
                                   document_encoder_path="facebook/contriever",
                                   batch_size=32)
tasb_search = Contriever(config_instance)
similarity_measure = CosScore()
response = tasb_search.retrieve(corpus, queries, 100, similarity_measure)
# print(response)
print("indices",len(response))

token_emb torch.Size([1200, 35, 768])
sentence_emb torch.Size([1200, 768])
indices 1200


In [9]:
from metrics.retrieval.RetrievalMetrics import RetrievalMetrics

metrics = RetrievalMetrics(k_values=[1,5,10])
print(metrics.evaluate_retrieval(qrels=qrels, results=response))

({'NDCG@1': 0.445, 'NDCG@5': 0.28297, 'NDCG@10': 0.21355}, {'MAP@1': 0.04475, 'MAP@5': 0.08901, 'MAP@10': 0.10608}, {'Recall@1': 0.04475, 'Recall@5': 0.11961, 'Recall@10': 0.16096}, {'P@1': 0.445, 'P@5': 0.23783, 'P@10': 0.16008})


In [10]:
# Understand queries
print(queries[0].id())
print(queries[0].text())

# Qrels
qrel1 = next(iter(qrels.items()))
test_query_id, test_context_ids = qrel1

print(test_query_id)
print(test_context_ids)

# Corpus
print(corpus[401226].text())

8813f87c0bdd11eba7f7acde48001122
Who is the mother of the director of film Polish-Russian War (Film)?
8813f87c0bdd11eba7f7acde48001122
{'314523': 1, '387778': 1, '550339': 1, '69975': 1, '126271': 1, '35800': 1, '369523': 1, '401226': 1, '237230': 1, '209617': 1}
Xawery Żuławski (born 22 December 1971 in Warsaw) is a Polish film director. In 1995 he graduated National Film School in Łódź. He is the son of actress Małgorzata Braunek and director Andrzej Żuławski. His second feature "Wojna polsko-ruska" (2009), adapted from the controversial best-selling novel by Dorota Masłowska, won First Prize in the New Polish Films competition at the 9th Era New Horizons Film Festival in Wrocław. In 2013, he stated he intends to direct a Polish novel "Zły" by Leopold Tyrmand. Żuławski and his wife Maria Strzelecka had 2 children together: son Kaj Żuławski (born 2002) and daughter Jagna Żuławska (born 2009).


In [ ]:
next(iter(response.items()))

In [13]:
import numpy as np

def expand_contexts(top_k=5, n_additional_docs=10, strategy="top_k_plus_hard_negative_docs"):
    q_contexts = {}
    
    for q_id, contexts in response.items():
        context_ids = list(map(int, contexts.keys()))
        q_contexts[q_id] = context_ids[:top_k]
        if strategy == "top_k_plus_random_docs":
            # Sample n_additional_docs random docs. If one of them is already in the top k, sample again
            while len(q_contexts[q_id]) < top_k + n_additional_docs:
                random_doc = np.random.choice(len(corpus), 1)[0]
                if random_doc not in context_ids:
                    q_contexts[q_id].append(random_doc)
                    
        elif strategy == "top_k_plus_hard_negative_docs":
            # Sample n_additional_docs hard negative docs from context_ids. A hard negative doc is a doc that is relevant to another query
            # but not to the current query and it is not present in the current query's ground truth
            gt_contexts = list(map(int, qrels[q_id].keys()))
            # Make a new list of context_ids that are not in the current query's ground truth. Use filter
            context_ids = list(filter(lambda x: x not in gt_contexts, context_ids))
            # Concat n_additional_docs hard negative docs
            q_contexts[q_id] += context_ids[:n_additional_docs]
            
    return q_contexts

In [14]:
# strategy = "top_k_plus_hard_negative_docs"
strategy = "top_k_plus_hard_negative_docs"
n_additional_docs = 10
top_k = 5

q_contexts = expand_contexts(top_k, n_additional_docs, strategy)
print(q_contexts)
# Save q_contexts to a file
np.save("exp_3_4/{}__top_k{}__add_docs_{}.npy".format(strategy, top_k, n_additional_docs), q_contexts)

{'8813f87c0bdd11eba7f7acde48001122': [454664, 126271, 258738, 83041, 474600, 454664, 258738, 83041, 474600, 96516, 420403, 401699, 373732, 185071, 269921], '61a46987092f11ebbdaeac1f6bf848b6': [458648, 143517, 250316, 404896, 70651, 136173, 404594, 93064, 486842, 301988, 4173, 142315, 561133, 452718, 120109], 'e2a3bf2a0bdd11eba7f7acde48001122': [193912, 346210, 81390, 273667, 409987, 346210, 409987, 246891, 491634, 290377, 374186, 431980, 140941, 7254, 529137], '0cd3bdea0bde11eba7f7acde48001122': [196475, 548430, 337684, 164155, 242223, 196475, 548430, 337684, 164155, 242223, 430896, 163305, 7066, 152580, 133073], 'f9dcb4a60bda11eba7f7acde48001122': [171617, 230191, 59045, 167958, 229646, 230191, 59045, 167958, 229646, 72426, 313266, 407251, 526265, 106999, 108623], '8981e1ce0bb011ebab90acde48001122': [150999, 530007, 259546, 482753, 493202, 530007, 493202, 140824, 259191, 473721, 557323, 369800, 26574, 168919, 85102], '019bed300bde11eba7f7acde48001122': [135801, 417200, 470046, 178480,

In [49]:
q_contexts_hn = q_contexts

np.save("exp_3_4/{}__top_k{}__add_docs_{}.npy".format("top_k_plus_hard_negative_docs", top_k, n_additional_docs), q_contexts_hn)

In [46]:
q_contexts_rd = expand_contexts(top_k, n_additional_docs, "top_k_plus_random_docs")

np.save("exp_3_4/{}__top_k{}__add_docs_{}.npy".format("top_k_plus_random_docs", top_k, n_additional_docs), q_contexts_rd)

In [50]:
from ollama import generate, create
import json
import tqdm

# Prompt construction
modelfile = """
FROM llama3:8b
PARAMETER temperature 0.5
"""

create(model="llama3-custom", modelfile=modelfile)


def create_prompt(question, context_ids):
    prompt = "Please answer the given question based on the given contexts below.\n"
    i = 1
    for id in context_ids:
        prompt += f"Context {i}: " + corpus[id].text() + "\n"
        i += 1
    prompt += "Question: " + question + "\n"
    prompt += "Constraint: Don't give any explanations and use MAX 5 tokens in your response. No yapping.\n"
    return prompt

def eval(retrived_contexts, test_data):
    print("Starting evaluation...")
    
    results = []
    exact_matches = 0
    ground_truth = ""
    question = ""
    i = 0
    for question_id, contexts_list in tqdm.tqdm(retrived_contexts.items()):
        context_ids = contexts_list
    
        for que in test_data:
            if que["_id"] == question_id:
                question = que["question"]
                ground_truth = que["answer"]
        prompt = create_prompt(question, context_ids)
        prediction = generate(model="llama3-custom", prompt=prompt)
        
        results.append({
            "q_id": question_id, 
            "prediction": prediction["response"], 
            "ground_truth": ground_truth
        })
        
        if prediction["response"] == ground_truth:
            exact_matches += 1
        
        # if i == 5:
        #     return results
        i+=1
            
    accuracy = exact_matches / len(test_data)
    results_info = {"exact_matches": exact_matches, "accuracy": accuracy}
    # with open(f"top_{top_k}.json", "w") as f:
    #     json.dump(results_info, f)
    print(results_info)
    return results

In [51]:
with open("dataset/data/dev.json", "r") as file:
    test_data = json.load(file)

# with open("dataset/wiki_musique_corpus.json", "r") as file:
#     context_data = json.load(file)

# with open("context_indices.json", "r") as file:
#     context_ids = json.load(file)

# Load q_contexts_hn
q_contexts_hn = np.load("exp_3_4/top_k_plus_hard_negative_docs__top_k5__add_docs_10.npy", allow_pickle=True).item()

# Load q_contexts_rd
q_contexts_rd = np.load("exp_3_4/top_k_plus_random_docs__top_k5__add_docs_10.npy", allow_pickle=True).item()

eval_results = eval(q_contexts_rd, test_data)
# eval(context_ids, context_data, test_data, 7)
# print(next(iter(context_ids.items())))

  0%|          | 5/1200 [01:09<4:35:56, 13.85s/it]


In [52]:
print(eval_results)

[{'q_id': '8813f87c0bdd11eba7f7acde48001122', 'prediction': 'Dorota Masłowska', 'ground_truth': 'Małgorzata Braunek'}, {'q_id': '61a46987092f11ebbdaeac1f6bf848b6', 'prediction': 'The Mysterious Dr. Fu Manchu (1929)', 'ground_truth': 'The Mask Of Fu Manchu'}, {'q_id': 'e2a3bf2a0bdd11eba7f7acde48001122', 'prediction': 'No information available.', 'ground_truth': '12 June 1516'}, {'q_id': '0cd3bdea0bde11eba7f7acde48001122', 'prediction': 'Best Music Director Award (Hindi)', 'ground_truth': 'Myanmar Motion Picture Academy Awards'}, {'q_id': 'f9dcb4a60bda11eba7f7acde48001122', 'prediction': 'Newfoundland', 'ground_truth': 'Missoula, Montana'}, {'q_id': '8981e1ce0bb011ebab90acde48001122', 'prediction': 'Charles Armand René de La Trémoille', 'ground_truth': 'Charles Armand René de La Trémoille'}]
